**Imports & Setup**

In [25]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

import pandas as pd
import numpy as np
import importlib
import src.simulate
importlib.reload(src.simulate)

from src.simulate import simulate_import_shock


**Load the two processed datasets**

In [2]:
PROJECT_ROOT = Path.cwd().parents[0] 
PROCESSED = PROJECT_ROOT / "data" / "processed"

prod = pd.read_parquet(PROCESSED / "production_filtered.parquet")
trade = pd.read_parquet(PROCESSED / "trade_filtered.parquet")

print(prod.shape)
print(trade.shape)

prod.head()

(8927, 6)
(19521, 7)


,country,country_code,commodity,commodity_code,year,production_qty
0,Afghanistan,2,Maize (corn),56,2000,115000.0
1,Afghanistan,2,Maize (corn),56,2001,160000.0
2,Afghanistan,2,Maize (corn),56,2002,298000.0
3,Afghanistan,2,Maize (corn),56,2003,210000.0
4,Afghanistan,2,Maize (corn),56,2004,400000.0


**Merge production & trade**

In [3]:
df = prod.merge(
    trade,
    on=["country", "country_code", "commodity", "commodity_code", "year"],
    how="left"
)

# if trade doesn't exist for some rows, treat as 0
df["import_qty"] = df["import_qty"].fillna(0)
df["export_qty"] = df["export_qty"].fillna(0)

df.head()

,country,country_code,commodity,commodity_code,year,production_qty,export_qty,import_qty
0,Afghanistan,2,Maize (corn),56,2000,115000.0,0.0,0.0
1,Afghanistan,2,Maize (corn),56,2001,160000.0,0.0,0.0
2,Afghanistan,2,Maize (corn),56,2002,298000.0,0.0,0.0
3,Afghanistan,2,Maize (corn),56,2003,210000.0,0.0,4.0
4,Afghanistan,2,Maize (corn),56,2004,400000.0,0.0,146.0


**Compute consumption & dependency ratio**

In [4]:
df["apparent_consumption"] = df["production_qty"] + df["import_qty"] - df["export_qty"]

# Remove impossible cases
df = df[df["apparent_consumption"] > 0].copy()

df["import_dependency_ratio"] = df["import_qty"] / df["apparent_consumption"]

#  Ensure ratio stays between 0 and 1
df["import_dependency_ratio"] = df["import_dependency_ratio"].clip(0, 1)

df[[
    "country", "commodity", "year",
    "production_qty", "import_qty", "export_qty",
    "apparent_consumption", "import_dependency_ratio"
]].head(20)

,country,commodity,year,production_qty,import_qty,export_qty,apparent_consumption,import_dependency_ratio
0,Afghanistan,Maize (corn),2000,115000.0,0.00,0.00,115000.00,0.000000
1,Afghanistan,Maize (corn),2001,160000.0,0.00,0.00,160000.00,0.000000
2,Afghanistan,Maize (corn),2002,298000.0,0.00,0.00,298000.00,0.000000
3,Afghanistan,Maize (corn),2003,210000.0,4.00,0.00,210004.00,0.000019
4,Afghanistan,Maize (corn),2004,400000.0,146.00,0.00,400146.00,0.000365
5,Afghanistan,Maize (corn),2005,315000.0,30.00,0.00,315030.00,0.000095
6,Afghanistan,Maize (corn),2006,359000.0,532.00,0.00,359532.00,0.001480
7,Afghanistan,Maize (corn),2007,360000.0,44.00,0.00,360044.00,0.000122
8,Afghanistan,Maize (corn),2008,360000.0,0.00,0.00,360000.00,0.000000
9,Afghanistan,Maize (corn),2009,300000.0,7589.00,73.00,307516.00,0.024678


**Create latest year snapshot ranking**

In [5]:
latest = (
    df.sort_values("year")
      .groupby(["country", "commodity"], as_index=False)
      .tail(1)
)

latest = latest.sort_values(["commodity", "import_dependency_ratio"], ascending=[True, False])

latest[["country", "commodity", "year", "import_dependency_ratio"]].head(30)

,country,commodity,year,import_dependency_ratio
3631,Latvia,Maize (corn),2023,1.000000
4114,Malta,Maize (corn),2024,1.000000
4032,Malaysia,Maize (corn),2024,1.000000
3882,Luxembourg,Maize (corn),2024,1.000000
3682,Lebanon,Maize (corn),2024,1.000000
4627,Netherlands (Kingdom of the),Maize (corn),2024,1.000000
3375,Jordan,Maize (corn),2024,1.000000
3150,Ireland,Maize (corn),2024,1.000000
574,Barbados,Maize (corn),2024,1.000000
6826,United Arab Emirates,Maize (corn),2024,1.000000


**Save base table**

In [6]:
base_out = PROCESSED / "base_country_commodity_year.parquet"
df.to_parquet(base_out, index=False)
base_out

WindowsPath('c:/projects/food-import-risk/data/processed/base_country_commodity_year.parquet')

**Prepare data**

In [7]:
# Remove regional aggregates so it's countries only
df = df[df["country_code"] < 5000].copy()

# Keep last 10 years for volatility calculations
recent = df[df["year"] >= df["year"].max() - 10].copy()

recent.head()

,country,country_code,commodity,commodity_code,year,production_qty,export_qty,import_qty,apparent_consumption,import_dependency_ratio
14,Afghanistan,2,Maize (corn),56,2014,316000.0,1735.98,32787.74,347051.76,0.094475
15,Afghanistan,2,Maize (corn),56,2015,316000.0,133.50,1166.00,317032.50,0.003678
16,Afghanistan,2,Maize (corn),56,2016,311646.0,4.01,7061.38,318703.37,0.022157
17,Afghanistan,2,Maize (corn),56,2017,173912.0,0.00,4424.00,178336.00,0.024807
18,Afghanistan,2,Maize (corn),56,2018,106670.0,0.00,16180.77,122850.77,0.131711


**Production & Import volatility**

In [8]:
vol = (
    recent
    .groupby(["country", "commodity"])
    .agg(
        prod_volatility=("production_qty", "std"),
        import_volatility=("import_qty", "std"),
        mean_consumption=("apparent_consumption", "mean"),
        mean_idr=("import_dependency_ratio", "mean")
    )
    .reset_index()
)

# Normalize volatility by scale
vol["prod_vol_norm"] = vol["prod_volatility"] / vol["mean_consumption"]
vol["import_vol_norm"] = vol["import_volatility"] / vol["mean_consumption"]

vol.head()

,country,commodity,prod_volatility,import_volatility,mean_consumption,mean_idr,prod_vol_norm,import_vol_norm
0,Afghanistan,Maize (corn),76726.465304,22244.926471,2.790642e+05,0.059236,0.274942,0.079713
1,Afghanistan,Wheat,588163.892806,231146.419522,4.999278e+06,0.094248,0.117650,0.046236
2,Albania,Maize (corn),13372.418755,14636.583516,4.653802e+05,0.152393,0.028734,0.031451
3,Albania,Wheat,29977.687156,66995.115942,4.247619e+05,0.410513,0.070575,0.157724
4,Algeria,Maize (corn),5303.764759,519466.447775,4.357156e+06,0.998273,0.001217,0.119221


**Risk Score**

In [9]:
risk = vol.copy()

risk["risk_score"] = (
    0.5 * risk["mean_idr"] +
    0.3 * risk["import_vol_norm"].fillna(0) +
    0.2 * risk["prod_vol_norm"].fillna(0)
)

risk["risk_score"] = risk["risk_score"].clip(0, 1)

risk["risk_band"] = pd.cut(
    risk["risk_score"],
    bins=[0, 0.33, 0.66, 1],
    labels=["Low", "Medium", "High"]
)

risk.sort_values("risk_score", ascending=False).head(20)

,country,commodity,prod_volatility,import_volatility,mean_consumption,mean_idr,prod_vol_norm,import_vol_norm,risk_score,risk_band
79,Djibouti,Maize (corn),0.454352,13051.650515,6858.685556,0.992926,0.000066,1.902938,1.000000,High
147,Latvia,Maize (corn),0.000000,97031.031542,55052.103333,1.000000,0.000000,1.762531,1.000000,High
241,Slovenia,Maize (corn),46909.562871,369486.078874,383017.938182,0.930069,0.122474,0.964670,0.778931,High
157,Luxembourg,Maize (corn),449.559797,3340.199059,4289.695000,1.000000,0.104800,0.778657,0.754557,High
109,Grenada,Maize (corn),64.203994,5905.653556,5768.597273,0.836046,0.011130,1.023759,0.727377,High
148,Latvia,Wheat,402382.958929,191055.234863,512605.893636,0.899382,0.784975,0.372714,0.718500,High
244,Somalia,Wheat,13.643111,42795.301046,40455.890000,0.800635,0.000337,1.057826,0.717733,High
203,Oman,Wheat,2749.086060,683328.648512,972838.399091,0.998387,0.002826,0.702407,0.710481,High
155,Lithuania,Maize (corn),24627.357682,102921.559367,157469.356364,0.893714,0.156395,0.653597,0.674215,High
96,Finland,Maize (corn),0.000000,23484.752959,41048.884000,1.000000,0.000000,0.572117,0.671635,High


**Save table**

In [10]:
risk_out = PROCESSED / "risk_index_latest.parquet"
risk.to_parquet(risk_out, index=False)
risk_out

WindowsPath('c:/projects/food-import-risk/data/processed/risk_index_latest.parquet')

**Run the shock on latest year snapshot**

In [14]:
# Build latest snapshot table
latest_base = (
    df.sort_values("year")
      .groupby(["country", "commodity"], as_index=False)
      .tail(1)
      .copy()
)

# Remove non-standard areas (regions / aggregates / special groupings)
bad_patterns = [
    r"\(kingdom of the\)",   # e.g., Netherlands (Kingdom of the)
    r"\bworld\b",
    r"\bafrica\b",
    r"\beurope\b",
    r"\basia\b",
    r"\bamerica\b",
    r"\boceania\b",
    r"\bcaribbean\b",
    r"\bcentral america\b",
    r"\bsouth america\b",
    r"\bnorthern europe\b",
    r"\bsouthern europe\b",
    r"\beastern europe\b",
    r"\bwestern europe\b",
]
pat = "|".join(bad_patterns)

latest_base = latest_base[
    ~latest_base["country"].astype(str).str.lower().str.contains(pat, regex=True, na=False)
].copy()

# Remove obvious territories
exclude_exact = {
    "Puerto Rico",
    "Réunion",
    "French Guiana",
}
latest_base = latest_base[~latest_base["country"].isin(exclude_exact)].copy()

# Keep only latest global year available in the dataset
latest_year = latest_base["year"].max()
latest_base = latest_base[latest_base["year"] == latest_year].copy()


**Save simulation output**

In [28]:
sim20 = simulate_import_shock(latest_base, shock_pct=0.20)
sim20["flag_zero_consumption_after_shock"] = sim20["consumption_shocked"].eq(0)
sim20_ranked = sim20.sort_values("shortfall_pct", ascending=False)

sim_out = PROCESSED / "shock_simulation_latest_importdrop20.parquet"
sim20_ranked.to_parquet(sim_out, index=False)
sim_out

WindowsPath('c:/projects/food-import-risk/data/processed/shock_simulation_latest_importdrop20.parquet')

In [29]:
sim20_ranked.loc[
    (sim20_ranked["country"] == "Malta") & (sim20_ranked["commodity"] == "Wheat"),
    ["import_dependency_ratio", "shortfall_pct", "idr_shocked", "apparent_consumption", "consumption_shocked"]
]

,import_dependency_ratio,shortfall_pct,idr_shocked,apparent_consumption,consumption_shocked
4139,1.0,0.2,1.0,4921.36,3937.088


In [48]:
risk = pd.read_parquet(PROCESSED / "risk_index_latest.parquet")

DROP_SUBSTRINGS = [
    ", mainland",
    "Taiwan Province of",
    "(Kingdom of the)"
]

def build_and_save_cached_sim(shock_pct: float):
    shock_key = int(round(shock_pct * 100)) 

    # run simulation
    sim = simulate_import_shock(latest_base, shock_pct=shock_pct)

    # flag
    sim["flag_zero_consumption_after_shock"] = sim["consumption_shocked"].eq(0)

    # shortfall_abs
    sim["shortfall_abs"] = (sim["apparent_consumption"] - sim["consumption_shocked"]).clip(lower=0)

    # remove special area or duplicate variants
    mask = False
    for s in DROP_SUBSTRINGS:
        mask = mask | sim["country"].str.contains(s, case=False, na=False, regex=False)
    sim = sim.loc[~mask].copy()

    # merge risk columns
    sim = sim.merge(
        risk[["country","commodity","risk_score","risk_band","mean_idr","prod_vol_norm","import_vol_norm"]],
        on=["country", "commodity"],
        how="left"
    )

    # no duplicates
    dup = int(sim.duplicated(subset=["country", "commodity"]).sum())
    print(f"shock={shock_pct:.2f} -> duplicates(country, commodity) = {dup}")

    # save
    out = PROCESSED / f"shock_simulation_latest_importdrop{shock_key}.parquet"
    sim.to_parquet(out, index=False)
    print("saved:", out)

    return sim


# Build all cached shocks
sim10 = build_and_save_cached_sim(0.10)
sim20 = build_and_save_cached_sim(0.20)
sim35 = build_and_save_cached_sim(0.35)
sim50 = build_and_save_cached_sim(0.50)

# Quick preview by absolute shortfall for 35%
sim35.sort_values("shortfall_abs", ascending=False)[
    ["country","commodity","shortfall_abs","shortfall_pct","risk_score","risk_band"]
].head(10)


shock=0.10 -> duplicates(country, commodity) = 0
saved: c:\projects\food-import-risk\data\processed\shock_simulation_latest_importdrop10.parquet
shock=0.20 -> duplicates(country, commodity) = 0
saved: c:\projects\food-import-risk\data\processed\shock_simulation_latest_importdrop20.parquet
shock=0.35 -> duplicates(country, commodity) = 0
saved: c:\projects\food-import-risk\data\processed\shock_simulation_latest_importdrop35.parquet
shock=0.50 -> duplicates(country, commodity) = 0
saved: c:\projects\food-import-risk\data\processed\shock_simulation_latest_importdrop50.parquet


,country,commodity,shortfall_abs,shortfall_pct,risk_score,risk_band
61,Mexico,Maize (corn),8.348812e+06,0.173374,0.222814,Low
44,China,Maize (corn),6.508380e+06,0.020745,0.051629,Low
228,Japan,Maize (corn),5.346843e+06,0.349996,0.508695,Medium
88,Egypt,Wheat,4.583548e+06,0.203763,0.301098,Low
43,China,Wheat,4.377393e+06,0.028684,0.039759,Low
176,Republic of Korea,Maize (corn),3.969589e+06,0.347275,0.520147,Medium
234,Italy,Wheat,3.292769e+06,0.212692,0.288354,Low
2,Spain,Maize (corn),3.283937e+06,0.258340,0.389960,Medium
175,Viet Nam,Maize (corn),3.084683e+06,0.235947,0.374945,Medium
89,Egypt,Maize (corn),2.761512e+06,0.183052,0.282625,Low


In [49]:
sorted([p.name for p in PROCESSED.glob("shock_simulation_latest_importdrop*.parquet")])

['shock_simulation_latest_importdrop10.parquet',
 'shock_simulation_latest_importdrop20.parquet',
 'shock_simulation_latest_importdrop35.parquet',
 'shock_simulation_latest_importdrop50.parquet']

In [46]:
df = pd.read_parquet(PROCESSED / "shock_simulation_latest_importdrop35.parquet")
print(df.columns.tolist())
print("risk_score exists:", "risk_score" in df.columns)
df.head()

['country', 'country_code', 'commodity', 'commodity_code', 'year', 'production_qty', 'export_qty', 'import_qty', 'apparent_consumption', 'import_dependency_ratio', 'consumption_shocked', 'shortfall_pct', 'idr_shocked', 'flag_zero_consumption_after_shock', 'shortfall_abs', 'risk_score', 'risk_band', 'mean_idr', 'prod_vol_norm', 'import_vol_norm']
risk_score exists: True


,country,country_code,commodity,commodity_code,year,production_qty,export_qty,import_qty,apparent_consumption,import_dependency_ratio,consumption_shocked,shortfall_pct,idr_shocked,flag_zero_consumption_after_shock,shortfall_abs,risk_score,risk_band,mean_idr,prod_vol_norm,import_vol_norm
0,Somalia,201,Wheat,15,2024,1063.3,0.00,4586.00,5649.30,0.811782,4.044200e+03,0.284124,0.737080,False,1.605100e+03,0.717733,High,0.800635,0.000337,1.057826
1,Somalia,201,Maize (corn),56,2024,69000.0,0.00,2849.40,71849.40,0.039658,7.085211e+04,0.013880,0.026141,False,9.972900e+02,0.128433,Low,0.096344,0.254127,0.098117
2,Spain,203,Maize (corn),56,2024,3500060.0,171074.37,9382676.71,12711662.34,0.738116,9.427725e+06,0.258340,0.646894,False,3.283937e+06,0.389960,Medium,0.681044,0.046274,0.133944
3,Slovenia,198,Wheat,15,2024,153390.0,394102.82,444707.76,203994.94,1.000000,1.325967e+05,0.350000,1.000000,False,7.139823e+04,0.642135,Medium,0.863254,0.057906,0.663091
4,Angola,7,Wheat,15,2024,37301.0,2.55,455213.76,492512.21,0.924269,3.331874e+05,0.323494,0.888056,False,1.593248e+05,0.583550,Medium,0.713807,0.032389,0.733894


In [47]:
sim35.sort_values("shortfall_abs", ascending=False)[
    ["country","commodity","shortfall_abs","shortfall_pct","risk_score","risk_band"]
].head(10)

,country,commodity,shortfall_abs,shortfall_pct,risk_score,risk_band
61,Mexico,Maize (corn),8.348812e+06,0.173374,0.222814,Low
44,China,Maize (corn),6.508380e+06,0.020745,0.051629,Low
228,Japan,Maize (corn),5.346843e+06,0.349996,0.508695,Medium
88,Egypt,Wheat,4.583548e+06,0.203763,0.301098,Low
43,China,Wheat,4.377393e+06,0.028684,0.039759,Low
176,Republic of Korea,Maize (corn),3.969589e+06,0.347275,0.520147,Medium
234,Italy,Wheat,3.292769e+06,0.212692,0.288354,Low
2,Spain,Maize (corn),3.283937e+06,0.258340,0.389960,Medium
175,Viet Nam,Maize (corn),3.084683e+06,0.235947,0.374945,Medium
89,Egypt,Maize (corn),2.761512e+06,0.183052,0.282625,Low
